In [3]:
import re
import json
import urllib2
import sys

In [23]:
def get_info_from_html(html):

    # finds a long digit
    idRE = re.compile(r"1(\d)(\d)(\d)+")
    if idRE.search(html) != None:
        page_index = idRE.search(html).group(0)

    # finds something in between >  <
    # BAD *$&@%$*&% will breka this
    labelRE = re.compile(r">[0-9A-z \'()-/]+<")
    if labelRE.search(html) != None:
        page_label = labelRE.search(html).group(0)
        page_label = page_label[1:-1]
    
    if 'page_index' in locals() and 'page_label' in locals():
        # url = 'http://www.mountainproject.com/v/' + page_index

        # TODO package back should be JSON
        return page_index, page_label #, url
    else:
        return None

In [33]:
def find_dest_iter(page_id, mp_html):
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(mp_html, 'html.parser')
    if page_id == '': # TODO move this to be the last case
        dest_iter = [str(s) for s in soup.find_all('span') if re.search("destArea",str(s)) != None]
    elif re.search("leftNavRoutes", mp_html) != None:
        t = str(soup.find(id="leftNavRoutes"))
        dest_iter = t.split("</td><td>")
    elif re.search('leftnavfilternote(.*)',mp_html) != None: # TODO Use beautiful soup
        nav_start = re.search('leftnavfilternote(.*)',mp_html).end()
        nav_end = re.search('(.*)LeftNavDynamicContent',mp_html).start()
        nav_html = mp_html[nav_start:nav_end]

        # break out location rows
        TRIM = 26
        dest_iter = nav_html[TRIM:].split('<br/>')
    else:
        dest_iter = None
    return dest_iter

In [24]:
def get_children(page_id):
    
    url = 'http://www.mountainproject.com/v/' + str(page_id)
    
    try:
        mp_page = urllib2.urlopen(url)
    except:
        return None
    else:
        mp_html = mp_page.read()
        
        website_tree = {}
        
        dest_iter = find_dest_iter(page_id, mp_html)
        
        if dest_iter != None:
            for dest in dest_iter:
                features = get_info_from_html(dest)

                if features != None:
                    page_index, page_label = features
                    # print page_label + " " + page_index
                    data = { 'page_id':page_index, 'label':page_label }
                    website_tree[page_index] = data
                else: # some bullshit
                    if re.search('leftnav',dest) == None: 
                        print dest # see if labelRE is missing anything
    
    c = { 'pageID':page_id }
    c['children_info'] = website_tree
    route_list = [c['children_info'][route]['page_id'] for route in c['children_info']]
    c['children'] = route_list

    if page_id in c['children']:
        return None
    else:
        return route_list

In [25]:
def get_info_box_html(mp_html):
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(mp_html, 'html.parser')
    page_table = soup.find_all('table')
    for t in page_table:
        if re.search("Submitted By:",str(t)) != None:
            return t

In [32]:
def get_route_info(mp_html):
    
    route_info = {}
    
    info_box_html = get_info_box_html(mp_html)
    
    if info_box_html != None:
        table_row = info_box_html.find_all('tr')
        for r in table_row:
            morsel = r.get_text()
            morsel = morsel.replace('\n','')
            i = morsel.split(u':\xa0')

            if str(i[0]) != 'Consensus':
                route_info[str(i[0])] = str(i[1])
            else:
                grade = r.get_text()[12:]
                for g in grade.split(u' '):
                    h = g.split(u':\xa0')
                    if len(h) > 1:
                        route_info['Consensus-'+str(h[0])] = str(h[1])

        return route_info
    else:
        return None

get_route_info()

TypeError: object of type 'int' has no len()

In [31]:
def traverse(tree, page_id):
    children = get_children(page_id)
    for child in children:
        if get_children(child) != None:
            traverse(tree, child) # RECURSION!!!
        else:
            tree = tree.append(get_route_info(page_id))
            return tree, child

tree, child = traverse([],'106861605')

TypeError: 'NoneType' object is not iterable

In [12]:
print tree

105746527


In [283]:
def get_info_box_html(mp_html):
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(mp_html, 'html.parser')
    page_table = soup.find_all('table')
    for t in page_table:
        if re.search("Submitted By:",str(t)) != None:
            return t

info_box_html = get_info_box(mp_html)
print info_box_html

<table cellpadding="0" cellspacing="0">
<tr>
<td>Type: </td>
<td>Boulder</td>
</tr>
<tr><td>Consensus: </td><td><span class="rateHueco"> <a class="small normal" href="#" onclick="showModalDlgAjax('/scripts/XmlHttpRequest.php?action=ratingOptions', 360, 'Rating Options');return false;">Hueco:</a> V5</span> <span class="rateFont"> <a class="small normal" href="#" onclick="showModalDlgAjax('/scripts/XmlHttpRequest.php?action=ratingOptions', 360, 'Rating Options');return false;">Font:</a> 6C</span> <a class="small" href="/scripts/ShowObjectStats.php?id=107752157">[details]</a></td></tr>
<tr><td valign="top">FA: </td><td><!--MPTEXT--><!--MPTEXTEND--></td></tr>
<tr><td>Page Views: </td><td>511</td></tr>
<tr><td nowrap="nowrap">Submitted By: </td><td><a href="/u/morgan-run-guide//107737661">Morgan Run Guide</a> on Aug 16, 2012</td></tr>
</table>


In [288]:
def get_route_info(mp_html):
    
    route_info = {}
    
    info_box_html = get_info_box_html(mp_html)
    
    table_row = info_box_html.find_all('tr')
    for r in table_row:
        morsel = r.get_text()
        morsel = morsel.replace('\n','')
        i = morsel.split(u':\xa0')

        if str(i[0]) != 'Consensus':
            route_info[str(i[0])] = str(i[1])
        else:
            grade = r.get_text()[12:]
            for g in grade.split(u' '):
                h = g.split(u':\xa0')
                if len(h) > 1:
                    route_info['Consensus-'+str(h[0])] = str(h[1])

    return route_info

get_route_info(mp_html)

{'Consensus-Font': '6C',
 'Consensus-Hueco': 'V5',
 'FA': '',
 'Page Views': '511',
 'Submitted By': 'Morgan Run Guide on Aug 16, 2012',
 'Type': 'Boulder'}

In [331]:
url = 'https://www.mountainproject.com/v/lone-star/105732662'
mp_page = urllib2.urlopen(url)
mp_html = mp_page.read()

soup = BeautifulSoup(mp_html, 'html.parser')

soup.find(itemprop="itemreviewed")
a = soup.find(id="starSummaryText").find_all('meta')
a

[<meta content="3.1" itemprop="average"/>,
 <meta content="4" itemprop="best"/>,
 <meta content="14" itemprop="votes"/>]

In [ ]:
len(tree)

In [57]:
url = 'https://www.mountainproject.com/v/' + str(107431192)
mp_page = urllib2.urlopen(url)
mp_html = mp_page.read()
from bs4 import BeautifulSoup
soup = BeautifulSoup(mp_html, 'html.parser')

#soup.find(id="routeSortLabel") != None

# def has_class_but_no_id(tag):
#     if tag.div and tag.has_attr('class'):
#         if u'medium' in tag['class']:
#             print tag

# soup.find_all(has_class_but_no_id)


route_name = soup.h1.get_text().replace(u'\xa0', '')
# class="dkorange"
print route_name

Hungry for Me


In [59]:
x = {'a': 1, 'b': 2}
y = {'b': 3, 'c': 4}

In [60]:
z = x.copy()
z.update(y)

In [61]:
z

{'a': 1, 'b': 3, 'c': 4}

In [68]:
x = {'Consensus-Ewbanks': '24', 'Consensus-British': 'E4', 'Consensus-French': '7a', 'FA': 'Charles Cole, John Middendorf, and Rusty Reno, 1985', 'Submitted By': 'Rusty Reno on Mar 31, 2007', 'Consensus-UIAA': 'VIII', 'Consensus-YDS': '5.11+', 'Consensus-ZA': '24', 'Type': "Trad, 12 pitches, 1400',  Grade III", 'Page Views': '14,828'}

In [69]:
y = { 'name': 'Stake 9' }

In [74]:
z = x.copy()
z.update(y)
print z


{'Consensus-Ewbanks': '24', 'Consensus-British': 'E4', 'Consensus-French': '7a', 'Consensus-YDS': '5.11+', 'Submitted By': 'Rusty Reno on Mar 31, 2007', 'name': 'Stake 9', 'Consensus-UIAA': 'VIII', 'FA': 'Charles Cole, John Middendorf, and Rusty Reno, 1985', 'Consensus-ZA': '24', 'Type': "Trad, 12 pitches, 1400',  Grade III", 'Page Views': '14,828'}
